# Aurora3
- 한국어 문장 감성분석
- 기존 단어들을 기반으로 같은 문장 내 다른 단어들의 감성을 측정
- 입력 데이터가 클수록 보다 정확한 감성분석이 가능

## 1. 데이터 불러오기

In [1]:
import pandas as pd
import re
import konlpy
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
from tqdm import tqdm
import urllib.request

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-2, 2))

In [65]:
# 입력 데이터프레임은 다음과 같은 양식으로 통일할 것
df = pd.read_table('ratings_train.txt')
df = df.iloc[:10000,:]
df = df[['id','document']]
df = df.rename(columns={"id": "id", "document": "sentence"})
df

,id,sentence
0,9976970,아 더빙.. 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
...,...,...
9995,8665166,곰티비로 무료로 봤기때문에 5점주려고했는데 1 한국 공포영화의 특징인 깜놀시키려 하...
9996,8312675,이딴걸드라마라고썼냐 수습할수없으면걍친자녀아니면되고 간단하네 얼굴을바꿨으면 결말이라도...
9997,6386483,왠지 김연아 크면 에리카처럼 될것같음.
9998,4452600,솔직히 굿 ㅋㅋㅋㅋ 넘버11씨는 살아남길 바랬는데 2번째극장판 어서 나오길


In [66]:
sent_dic = pd.read_csv('SentiWord_Dict.txt',sep = '\t',header=None)
sent_dic.iloc[14850,0]='갈등'

pos_dic = sent_dic[sent_dic[1]>0]
neg_dic = sent_dic[sent_dic[1]<0]
neu_dic = sent_dic[sent_dic[1]==0]

## 2. 텍스트 전처리

In [67]:
okt = konlpy.tag.Okt()

def text_preprocess(x):
    text=[]
    a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

def tokenize(x):
    text = []
    tokens = okt.pos(x)
    for token in tokens :
        if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
            text.append(token[0])
    return text

In [68]:
tqdm.pandas()
df['comment'] = df['sentence'].apply(lambda x : text_preprocess(x))
df['comment'] = df['comment'].progress_apply(lambda x: tokenize(x))
df

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:40<00:00, 245.91it/s]


,id,sentence,comment
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,"[아, 더빙, 진짜, 짜증나네요, 목소리]"
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,"[흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 조차, 가볍지, 않구나]"
2,10265843,너무재밓었다그래서보는것을추천한다,"[너, 무재, 밓었, 다그, 래서, 보는것을, 추천, 한, 다]"
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,"[교도소, 이야기, 구먼, 솔직히, 재미, 는, 없다, 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,"[사이, 몬페, 그, 의, 익살스런, 연기, 가, 돋보였던, 영화, 스파이더맨, 에..."
...,...,...,...
9995,8665166,곰티비로 무료로 봤기때문에 5점주려고했는데 1 한국 공포영화의 특징인 깜놀시키려 하...,"[곰, 티비, 로, 무료, 로, 봤기, 때문, 에, 5, 점주, 려고, 했는데, 1..."
9996,8312675,이딴걸드라마라고썼냐 수습할수없으면걍친자녀아니면되고 간단하네 얼굴을바꿨으면 결말이라도...,"[이딴, 걸, 드라마, 라고, 썼냐, 수습, 할수, 없으면, 걍, 친, 자녀, 아니..."
9997,6386483,왠지 김연아 크면 에리카처럼 될것같음.,"[왠지, 김연아, 크면, 에리, 카, 처럼, 될것, 같음]"
9998,4452600,솔직히 굿 ㅋㅋㅋㅋ 넘버11씨는 살아남길 바랬는데 2번째극장판 어서 나오길,"[솔직히, 굿, 넘버, 11, 씨, 는, 살아남길, 바랬는데, 2, 번, 째, 극장..."


## 3. 단어별 감성점수 측정

In [71]:
def make_sent_dict(x) :
    pos=[]
    neg=[]
    tmp={}
    
    for sentence in tqdm(x):
        for word in sentence :
            target = sent_dic[sent_dic[0]==word]
            if len(target)==1: # 기존에 있는 단어라면 그대로 사용
                score = float(target[1])
                if score > 0:
                    pos.append(word)
                elif score < 0:
                    neg.append(word)                
            tmp[word] = {'W':0,'WP':0,'WN':0} # 감성사전 구성
    pos = list(set(pos))
    neg = list(set(neg))
    
    for sentence in tqdm(x):
        for word in sentence :
            tmp[word]['W'] += 1 # 빈도 수
            for po in pos :
                if po in sentence:
                    tmp[word]['WP'] += 1 # 긍정단어과 같은 문장 내 단어일 때
                    break
            for ne in neg:
                if ne in sentence:
                    tmp[word]['WN'] += 1 # 부정단어와 같은 문장내 단어일 때
                    break
    return pos, neg, pd.DataFrame(tmp)

In [72]:
pos, neg, new_dict = make_sent_dict(df['comment'].values)
new_dict

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:35<00:00, 282.73it/s]


,아,더빙,진짜,짜증나네요,목소리,흠,포스터,보고,초딩,영화,...,아니면되,간단하네,토나올것,먹는다,유치하고어이가,쓰지마라,김연아,크면,에리,살아남길
W,265,25,546,3,20,25,51,329,27,3352,...,1,1,1,1,1,1,1,1,1,1
WP,66,2,189,1,6,5,10,136,7,1362,...,1,1,1,1,1,1,0,0,0,0
WN,89,8,202,1,4,8,15,96,5,1012,...,1,1,1,1,1,1,0,0,0,0


In [73]:
def make_score_dict(d,p,n):
    N=sum(d.iloc[0,::])
    pos_cnt=sum(d.loc[::,p].iloc[0,::])
    neg_cnt=sum(d.loc[::,n].iloc[0,::])
    
    trans =d.T
    trans['neg_cnt']=neg_cnt
    trans['pos_cnt']=pos_cnt
    trans['N']=N

    trans['MI_P']=np.log2(trans['WP']*trans['N']/trans['W']*trans['pos_cnt'])
    trans['MI_N']=np.log2(trans['WN']*trans['N']/trans['W']*trans['neg_cnt'])
    trans['SO_MI']=trans['MI_P'] - trans['MI_N']
    
    trans = trans.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    trans = trans.sort_values(by=['SO_MI'],ascending=False)
    return trans

In [74]:
t_dict = make_score_dict(new_dict,pos,neg)
t_dict['SO_MI'] = scaler.fit_transform(t_dict['SO_MI'].values.reshape(-1,1))
t_dict

c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


,W,WP,WN,neg_cnt,pos_cnt,N,MI_P,MI_N,SO_MI
꿀잼,21,21,1,3244,3987,130430,28.954004,24.264157,2.000000
멋지다,17,17,1,3244,3987,130430,28.954004,24.569011,1.852197
굿,64,17,1,3244,3987,130430,27.041467,22.656474,1.852197
존경,13,13,1,3244,3987,130430,28.954004,24.956035,1.664556
열정,13,13,1,3244,3987,130430,28.954004,24.956035,1.664556
...,...,...,...,...,...,...,...,...,...
슬프다,10,1,10,3244,3987,130430,25.632076,28.656474,-1.740105
미친,33,3,33,3244,3987,130430,25.494572,28.656474,-1.806771
지루,12,1,12,3244,3987,130430,25.369042,28.656474,-1.867632
발연기,39,3,39,3244,3987,130430,25.253564,28.656474,-1.923619


In [75]:
def update_dict(d):
    add_Dic = {0:[],1:[]}
    for i in d.T.items():
        if i[0] not in list(sent_dic[0]):
            if len(i[0]) > 1:
                add_Dic[0].append(i[0])
                add_Dic[1].append(i[1]['SO_MI'])
            
    add_Dic=pd.DataFrame(add_Dic)
    Sentiment=pd.merge(sent_dic,add_Dic,'outer')
    return Sentiment

In [76]:
add_dict =update_dict(t_dict)
add_dict

,0,1
0,(-;,1.000000
1,(;_;),-1.000000
2,(^^),1.000000
3,(^-^),1.000000
4,(^^*,1.000000
...,...,...
20508,카메라,-1.382801
20509,기만,-1.490624
20510,나온다,-1.584024
20511,자살,-1.584024


## 4. 문장 감성분석

In [77]:
def get_cnt(x):
    cnt = 0
    for word in list(set(x)):
        target = add_dict[add_dict[0]==word]
        if len(target)==1:
            cnt += float(target[1])
    return cnt

def get_ratio(x):
    score = x['score']
    length = np.log(len(x['comment']))+1
    try:
        ratio= round(score/length,2)
    except:
        ratio = 0
    return ratio

In [78]:
tqdm.pandas()
df['score']= df['comment'].progress_apply(lambda x : get_cnt(x))
df['ratio'] = df.apply(lambda x: get_ratio(x), axis = 1)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [07:23<00:00, 22.53it/s]
c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


In [79]:
df

,id,sentence,comment,score,ratio
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,"[아, 더빙, 진짜, 짜증나네요, 목소리]",-1.250708,-0.48
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,"[흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 조차, 가볍지, 않구나]",-0.029085,-0.01
2,10265843,너무재밓었다그래서보는것을추천한다,"[너, 무재, 밓었, 다그, 래서, 보는것을, 추천, 한, 다]",0.672279,0.21
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,"[교도소, 이야기, 구먼, 솔직히, 재미, 는, 없다, 평점, 조정]",0.558321,0.17
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,"[사이, 몬페, 그, 의, 익살스런, 연기, 가, 돋보였던, 영화, 스파이더맨, 에...",1.773512,0.44
...,...,...,...,...,...
9995,8665166,곰티비로 무료로 봤기때문에 5점주려고했는데 1 한국 공포영화의 특징인 깜놀시키려 하...,"[곰, 티비, 로, 무료, 로, 봤기, 때문, 에, 5, 점주, 려고, 했는데, 1...",-9.285466,-1.85
9996,8312675,이딴걸드라마라고썼냐 수습할수없으면걍친자녀아니면되고 간단하네 얼굴을바꿨으면 결말이라도...,"[이딴, 걸, 드라마, 라고, 썼냐, 수습, 할수, 없으면, 걍, 친, 자녀, 아니...",-3.242275,-0.64
9997,6386483,왠지 김연아 크면 에리카처럼 될것같음.,"[왠지, 김연아, 크면, 에리, 카, 처럼, 될것, 같음]",-0.167852,-0.05
9998,4452600,솔직히 굿 ㅋㅋㅋㅋ 넘버11씨는 살아남길 바랬는데 2번째극장판 어서 나오길,"[솔직히, 굿, 넘버, 11, 씨, 는, 살아남길, 바랬는데, 2, 번, 째, 극장...",-0.991101,-0.27


In [86]:
df[(df['ratio'] > 2) ]

,id,sentence,comment,score,ratio
276,9077483,정은지 언니!! 연기 잘하구..노래도 잘부르시고! 마지막회 웃으면서 즐겁게! 봤습니...,"[정은지, 언니, 연기, 잘, 하구, 노래, 도, 잘, 부르시고, 마지막, 회, 웃...",8.061200,2.10
466,501464,귀여운벤지.. 똑똑한 벤지 예쁘고 귀여운 친구... 압삘럽 ㅎ,"[귀여운, 벤지, 똑똑한, 벤지, 예쁘고, 귀여운, 친구, 압삘럽]",7.000000,2.27
2641,8311882,화려한 영상미를 뽐내는 현대판 천녀유혼! 아주 잘봤습니다. 매우만족!,"[화려한, 영, 상미, 를, 뽐내는, 현, 대판, 천녀유혼, 아주, 잘, 봤습니다,...",7.819670,2.19
2855,7081343,따뜻하고 좋은 영화네요. 감동적입니다!,"[따뜻하고, 좋은, 영화, 네, 요, 감동, 적, 입니다]",6.513808,2.12
3883,9252218,어릴때도 참 재밌게 봤는데 커서 봐서도 하나도 유치하지않고 스토리 구성도 탄탄하고 ...,"[어릴, 때, 도, 참, 재밌게, 봤는데, 커서, 봐서도, 하나, 도, 유치하지, ...",10.926987,2.22
4296,6715139,무언가 슬프지만 가슴이 따뜻한 영화입니다. 어두운 현실이지만 밝은 풍경과 예쁜 마음...,"[무언가, 슬프지만, 가슴, 이, 따뜻한, 영화, 입니다, 어, 두운, 현실, 이지...",11.344598,2.48
5240,8215530,감동이에요 잘봤습니다,"[감동, 이에요, 잘, 봤습니다]",5.037565,2.11
5517,9835979,오랜만에 좋은 애니봤습니다! 기차를.타고 떠나는 장면이 너무나 아름답고 좋았습니다!,"[오랜, 만, 에, 좋은, 애니, 봤습니다, 기차, 를, 타고, 떠나는, 장면, 이...",7.520039,2.03
5890,5236456,최고 예요 정말 감동했어요.,"[최고, 예요, 정말, 감동, 했어요]",5.361557,2.05
7047,9736077,신나게 웃고 즐기고 마지막 감동까지있는 요즘 본 영화중에 감명받게 볼수있던영화네요 ...,"[신나게, 웃고, 즐기고, 마지막, 감동, 까지, 있는, 요즘, 본, 영화, 중, ...",11.810645,2.64


## 5. 클래스

In [59]:
class Aurora3:
    
    def __init__(self, df,sent_dic):
        self.df = df
        self.okt = konlpy.tag.Okt()
        self.sent_dic = sent_dic
        
    def get_df(self):# 최종 결과 반환
        print("문장을 토큰화 중입니다...")
        self.tokenizer_run()
        
        print("감성사전을 업데이트 중입니다...")
        self.expand_sent_dic()
        
        print("문장 감성분석 중입니다....")
        self.sent_analyze()
        return self.df
        
    def tokenizer_run(self): # 텍스트 전처리 & 토큰화
        tqdm.pandas()
        
        def text_preprocess(x):
            text=[]
            a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
            for j in a.split():
                text.append(j)
            return ' '.join(text)

        def tokenize(x):
            text = []
            tokens = self.okt.pos(x)
            for token in tokens :
                if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
                    text.append(token[0])
            return text
        self.df['comment'] = self.df['sentence'].apply(lambda x : text_preprocess(x))
        self.df['comment'] = self.df['comment'].progress_apply(lambda x: tokenize(x))
    
    def expand_sent_dic(self):
        sent_dic = self.sent_dic
        
        def make_sent_dict(x) :
            pos=[]
            neg=[]
            tmp={}

            for sentence in tqdm(x):
                for word in sentence :
                    target = sent_dic[sent_dic[0]==word]
                    if len(target)==1: # 기존에 있는 단어라면 그대로 사용
                        score = float(target[1])
                        if score > 0:
                            pos.append(word)
                        elif score < 0:
                            neg.append(word)                
                    tmp[word] = {'W':0,'WP':0,'WN':0} # 감성사전 구성
            pos = list(set(pos))
            neg = list(set(neg))

            for sentence in tqdm(x):
                for word in sentence :
                    tmp[word]['W'] += 1 # 빈도 수
                    for po in pos :
                        if po in sentence:
                            tmp[word]['WP'] += 1 # 긍정단어과 같은 문장 내 단어일 때
                            break
                    for ne in neg:
                        if ne in sentence:
                            tmp[word]['WN'] += 1 # 부정단어와 같은 문장내 단어일 때
                            break
            return pos, neg, pd.DataFrame(tmp)
        
        def make_score_dict(d,p,n):
            N=sum(d.iloc[0,::])
            pos_cnt=sum(d.loc[::,p].iloc[0,::])
            neg_cnt=sum(d.loc[::,n].iloc[0,::])

            trans =d.T
            trans['neg_cnt']=neg_cnt
            trans['pos_cnt']=pos_cnt
            trans['N']=N

            trans['MI_P']=np.log2(trans['WP']*trans['N']/trans['W']*trans['pos_cnt'])
            trans['MI_N']=np.log2(trans['WN']*trans['N']/trans['W']*trans['neg_cnt'])
            trans['SO_MI']=trans['MI_P'] - trans['MI_N']

            trans = trans.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
            trans = trans.sort_values(by=['SO_MI'],ascending=False)
            return trans
        
        def update_dict(d):
            add_Dic = {0:[],1:[]}
            for i in d.T.items():
                if i[0] not in list(sent_dic[0]):
                    if len(i[0]) > 1:
                        add_Dic[0].append(i[0])
                        add_Dic[1].append(i[1]['SO_MI'])

            add_Dic=pd.DataFrame(add_Dic)
            Sentiment=pd.merge(sent_dic,add_Dic,'outer')
            return Sentiment
        
        self.pos, self.neg, self.new_dict = make_sent_dict(self.df['comment'].values)
        
        self.t_dict = make_score_dict(self.new_dict,self.pos,self.neg)
        self.t_dict['SO_MI'] = scaler.fit_transform(self.t_dict['SO_MI'].values.reshape(-1,1))
       
        self.add_dict =update_dict(self.t_dict)
    
    def sent_analyze(self): # 데이터 감성분석
        tqdm.pandas()
        
        def get_cnt(x):
            cnt = 0
            for word in list(set(x)):
                target = self.add_dict[self.add_dict[0]==word]
                if len(target)==1:
                    cnt += float(target[1])
            return cnt

        def get_ratio(x):
            score = x['score']
            length = np.log(len(x['comment']))+1
            try:
                ratio= round(score/length,2)
            except:
                ratio = 0
            return ratio
        
        tqdm.pandas()
        self.df['score']= self.df['comment'].progress_apply(lambda x : get_cnt(x))
        self.df['ratio'] = self.df.apply(lambda x: get_ratio(x), axis = 1)

In [60]:
test = Aurora3(df,sent_dic)
res = test.get_df()

  2%|█▎                                                                             | 17/1000 [00:00<00:05, 164.25it/s]

문장을 토큰화 중입니다...


  0%|▎                                                                                | 4/1000 [00:00<00:26, 37.97it/s]

감성사전을 업데이트 중입니다...


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 734.05it/s]
c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
  1%|▍                                                                                | 6/1000 [00:00<00:16, 59.32it/s]

문장 감성분석 중입니다....


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:25<00:00, 39.83it/s]
c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:123: RuntimeWarning: divide by zero encountered in log


In [61]:
res

,id,sentence,comment,score,ratio
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,"[아, 더빙, 진짜, 짜증나네요, 목소리]",-0.109437,-0.04
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,"[흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 조차, 가볍지, 않구나]",-0.520111,-0.15
2,10265843,너무재밓었다그래서보는것을추천한다,"[너, 무재, 밓었, 다그, 래서, 보는것을, 추천, 한, 다]",-0.109437,-0.03
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,"[교도소, 이야기, 구먼, 솔직히, 재미, 는, 없다, 평점, 조정]",0.372458,0.12
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,"[사이, 몬페, 그, 의, 익살스런, 연기, 가, 돋보였던, 영화, 스파이더맨, 에...",0.708715,0.18
...,...,...,...,...,...
995,10275296,미달이는.. 연예계를 바라보는 태도를 고쳐야 한다.,"[미달, 이, 는, 연예계, 를, 바라보는, 태도, 를, 고쳐야, 한다]",-0.218874,-0.07
996,6473973,파괴된 관객들.,"[파괴, 된, 관객, 들]",-2.000000,-0.84
997,8417568,알콜중독자들 덕에 평점이 낮은가?,"[알콜중독자, 들, 덕, 에, 평점, 이, 낮은가]",1.956763,0.66
998,7126211,중견배우들이 만들어놓은 긴장감이 나름 긴장감있게 싸워보려했던 도술사들에 의해 헛웃음...,"[중견, 배우, 들, 이, 만들어놓은, 긴장감, 이, 나름, 긴장감, 있게, 싸워,...",0.896179,0.22
